In [6]:
import cv2
import mediapipe as mp
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from keras.utils import to_categorical

import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.data import Dataset

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [7]:
def trim(df, max_samples, min_samples, column):
    df=df.copy()
    groups=df.groupby(column)    
    trimmed_df = pd.DataFrame(columns = df.columns)
    groups=df.groupby(column)
    for label in df[column].unique(): 
        group=groups.get_group(label)
        count=len(group)    
        if count > max_samples:
            sampled_group=group.sample(n=max_samples, random_state=123,axis=0)
            trimmed_df=pd.concat([trimmed_df, sampled_group], axis=0)
        else:
            if count>=min_samples:
                sampled_group=group        
                trimmed_df=pd.concat([trimmed_df, sampled_group], axis=0)
    print('after trimming, the maximum samples in any class is now ',max_samples, ' and the minimum samples in any class is ', min_samples)
    return trimmed_df

In [8]:
def show_image_samples(gen):
    t_dict = gen.class_indices
    classes = list(t_dict.keys())
    images, labels = next(gen)
    
    plt.figure(figsize = (20, 20))
    length = len(labels)
    
    if length<25:   #show maximum of 25 images
        r=length
    else:
        r=25
    
    for i in range(r):        
        plt.subplot(5, 5, i + 1)
        image=images[i] /255       
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color='blue', fontsize=12)
        plt.axis('off')
    plt.show()

In [13]:
def load_dataset(train_path, test_path, max_samples, min_samples, img_size, batch_size,
                    horizontal_flip=True, rotation_range=25, width_shift_range=.25,
                                height_shift_range=.3, zoom_range=.4):
    train_path = train_path
    test_path = test_path

    for d in [train_path, test_path]:
        filepaths = []
        labels = []
        classlist = sorted(os.listdir(d))

        for klass in classlist:        
            classpath = os.path.join(d, klass)
            flist = sorted(os.listdir(classpath))

            for f in flist:
                fpath = os.path.join(classpath,f)
                filepaths.append(fpath)
                labels.append(klass)

        Fseries = pd.Series(filepaths, name='filepaths')
        Lseries = pd.Series(labels, name='labels') 

        if d == train_path:            
            df = pd.concat([Fseries, Lseries], axis=1)
        else:
            test_df = pd.concat([Fseries, Lseries], axis=1)

    train_df, valid_df = train_test_split(df, train_size=.9, shuffle=True, random_state=123, stratify=df['labels']) 

    # get the number of classes and the images count for each class in train_df
    classes = sorted(list(train_df['labels'].unique()))
    class_count = len(classes)

    groups = train_df.groupby('labels')


    countlist = []
    classlist = []

    for label in sorted(list(train_df['labels'].unique())):
        group = groups.get_group(label)
        countlist.append(len(group))
        classlist.append(label)

    # get the classes with the minimum and maximum number of train images
    max_value = np.max(countlist)
    max_index = countlist.index(max_value)
    max_class = classlist[max_index]
    min_value = np.min(countlist)
    min_index = countlist.index(min_value)
    min_class = classlist[min_index]


    # lets get the average height and width of a sample of the train images
    ht = 0
    wt = 0

    # select 100 random samples of train_df
    train_df_sample = train_df.sample(n=100, random_state=123,axis=0)

    for i in range (len(train_df_sample)):
        fpath = train_df_sample['filepaths'].iloc[i]
        img = plt.imread(fpath)
        shape = img.shape
        ht += shape[0]
        wt += shape[1]
        
        
        
    max_samples = max_samples
    min_samples = min_samples
    column='labels'
    train_df= trim(train_df, max_samples, min_samples, column)  
        
        
    
    working_dir=r'./'
    img_size = img_size
    batch_size = batch_size

    trgen = ImageDataGenerator(horizontal_flip=horizontal_flip, rotation_range=rotation_range, width_shift_range=width_shift_range,
                                height_shift_range=height_shift_range, zoom_range=zoom_range)
    t_and_v_gen = ImageDataGenerator()

    msg = '{0:70s} for train generator'.format(' ')
    print(msg, '\r', end = '') # prints over on the same line

    train_gen = trgen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                       class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)

    msg = '{0:70s} for valid generator'.format(' ')
    print(msg, '\r', end='') # prints over on the same line

    valid_gen = t_and_v_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                       class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)


    # for the test_gen we want to calculate the batch size and test steps such that batch_size X test_steps= number of samples in test set
    # this insures that we go through all the sample in the test set exactly once.
    length = len(test_df)
    test_batch_size = sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]  
    test_steps = int(length/test_batch_size)

    msg = '{0:70s} for test generator'.format(' ')
    print(msg, '\r', end='') # prints over on the same line

    test_gen = t_and_v_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                       class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=test_batch_size)

    # from the generator we can get information we will need later
    classes = list(train_gen.class_indices.keys())
    class_indices = list(train_gen.class_indices.values())
    class_count = len(classes)
    labels = test_gen.labels

    print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps, ' number of classes : ', class_count)
    
    show_image_samples(train_gen)
    
    return train_gen, test_gen, valid_gen

In [14]:
train_path = 'D:\Projekat 2022\dataset\Synthetic ASL Alphabet\Train_Alphabet'
test_path = 'D:\Projekat 2022\dataset\Synthetic ASL Alphabet\Test_Alphabet'

In [2]:
def generate_keypoint_df(generator, num_of_samples):
    
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands = 1, static_image_mode=True, min_detection_confidence = 0.3)
    mpDraw = mp.solutions.drawing_utils
    
    x = np.zeros((num_of_samples, 21))
    y = np.zeros((num_of_samples, 21))
    z = np.zeros((num_of_samples, 21))
    labels = []
    
    samples = 0
    
    for batch in generator:
        for i in range(batch[0].shape[0]):
            img = batch[0][i].astype('uint8')
            label = batch[1][i] 
            
            results = hands.process(img)
            
            if results.multi_hand_landmarks:
                for handLms in results.multi_hand_landmarks:
                    
                    landmarks_x = np.zeros(21)
                    landmarks_y = np.zeros(21)
                    landmarks_z = np.zeros(21)
                    
                    for id, lm in enumerate(handLms.landmark):                           
                            
                        landmarks_x[id] = lm.x
                        landmarks_y[id] = lm.y
                        landmarks_z[id] = lm.z
                        
                x[samples] = landmarks_x
                y[samples] = landmarks_y
                z[samples] = landmarks_z
                labels.append(label)
                samples += 1
                b = str(samples)
                sys.stdout.write('\r'+ "Generated  " + b + "  samples out of  " + str(num_of_samples) + "  samples")

                
                if samples >= num_of_samples:
                    break

        if samples >= num_of_samples:
            break
    
    sys.stdout.write('\r'+ "DONE: Generated  " + b + "  samples")
    exit_labels = np.array(labels)
    
    return x, y, z, exit_labels

In [4]:
def generate_keypoint_sequential(generator, num_of_samples, include_z = False):
    
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands = 1, static_image_mode=True, min_detection_confidence = 0.3)
    mpDraw = mp.solutions.drawing_utils
 
    n = 2
    if include_z:
        n = 3
    
    data = np.zeros((num_of_samples, 21 * n))
    labels = []
    
    samples = 0
    
    for batch in generator:
        for i in range(batch[0].shape[0]):
            img = batch[0][i].astype('uint8')
            label = batch[1][i] 
            
            results = hands.process(img)
            
            if results.multi_hand_landmarks:
                for handLms in results.multi_hand_landmarks:
                    
                    i = 0
                    
                    for id, lm in enumerate(handLms.landmark):
                        
                        data[samples][i] = lm.x
                        i += 1
                        data[samples][i] = lm.y
                        i += 1
                        if include_z: 
                            data[samples][i] = lm.y
                            i += 1
                            
                labels.append(label)
                samples += 1
                message = str(samples)
                sys.stdout.write('\r'+ "Generated  " + message + "  samples out of  " + str(num_of_samples) + "  samples")
                #print("Generated  ", samples, "  samples")
                
                if samples >= num_of_samples:
                    break

        if samples >= num_of_samples:
            break
    
    sys.stdout.write('\r'+ "DONE: Generated  " + message + "  samples")
    exit_labels = np.array(labels)
    
    return data, exit_labels

In [17]:
import functools
def combine_dims(a, i=0, n=1):
  """
  Combines dimensions of numpy array `a`, 
  starting at index `i`,
  and combining `n` dimensions
  """
  s = list(a.shape)
  combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
  return np.reshape(a, s[:i] + [combined] + s[i+n+1:])

In [18]:
def make_keypoint_database(num_of_train_samples, num_of_test_samples, num_of_valid_samples):
    
    num_of_train_samples = num_of_train_samples
    num_of_test_samples = num_of_test_samples
    num_of_valid_samples = num_of_valid_samples
    
    new_train_df = generate_keypoint_df(train_gen, num_of_train_samples)
    new_test_df = generate_keypoint_df(train_gen, num_of_test_samples)
    new_valid_df = generate_keypoint_df(train_gen, num_of_valid_samples)
    
    train_db = new_train_df
    test_db = new_test_df
    valid_db = new_valid_df
    
    train_x = np.array([train_db[0], train_db[1]])
    train_y = np.array(train_db[3])
    test_x = np.array([test_db[0], test_db[1]])
    test_y = np.array(test_db[3])
    valid_x = np.array([valid_db[0], valid_db[1]])
    valid_y = np.array(valid_db[3])
    
    train_x = np.transpose(train_x, (1, 0, 2))
    test_x = np.transpose(test_x, (1, 0, 2))
    valid_x = np.transpose(valid_x, (1, 0, 2))
    
    train_x = combine_dims(train_x, 1)
    test_x = combine_dims(test_x, 1)
    valid_x = combine_dims(valid_x, 1)
    
    return train_x, train_y, test_x, test_y, valid_x, valid_y

In [19]:
def show_image_samples_with_landmarks(gen):
    t_dict = gen.class_indices
    classes = list(t_dict.keys())
    images, labels = next(gen)
    
    plt.figure(figsize = (20, 20))
    length = len(labels)
    
    if length<25:   #show maximum of 25 images
        r=length
    else:
        r=25
    
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands = 1, static_image_mode=True, min_detection_confidence = 0.3)
    mpDraw = mp.solutions.drawing_utils
    
    for i in range(r):        
        plt.subplot(5, 5, i + 1)
        image=images[i].astype('uint8')
        
        results = hands.process(image)
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                mpDraw.draw_landmarks(image, handLms, mpHands.HAND_CONNECTIONS)

        
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color='blue', fontsize=12)
        plt.axis('off')
    plt.show()

In [23]:
def zapisi_rezultate(METOD_NUM, EKSP_NUM, acc, loss, val_acc, val_loss):

    METOD_NUM = METOD_NUM
    EKSP_NUM = EKSP_NUM

    txt_file_path = "D:/Projekat 2022/Metod " + str(METOD_NUM) + "/Merenja/eksp_" + str(EKSP_NUM) + "/"
    files = [(acc, 'acc.txt'), (loss, 'loss.txt'), (val_acc, 'val_acc.txt'), (val_loss, 'val_loss.txt')]

    for data, file_name in files:

        textfile = open(txt_file_path + file_name, "w")

        #Brise sve iz fajla
        with textfile as file:
            pass

        textfile = open(txt_file_path + file_name, "w")

        for element in data:
            textfile.write(str(element) + file_name + "\n")
        textfile.close()

    subject = 'model_eksp_' + str(EKSP_NUM) 
    save_id = subject + '.h5' 
    model_save_loc = os.path.join(txt_file_path, save_id)
    model.save(model_save_loc)
    print ('model was saved as ' , model_save_loc ) 

In [25]:
def plot_loss_and_acc(history):

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'y', label = 'Training loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    acc_is_called = list(history.history.keys())[1]

    acc = history.history[acc_is_called]
    val_acc = history.history['val_' + acc_is_called]

    plt.plot(epochs, acc, 'y', label = 'Training acc')
    plt.plot(epochs, val_acc, 'r', label = 'Validation acc')
    plt.title('Training and validation acc')
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.legend()
    plt.show()